In [3]:
import import_ipynb
from math import sqrt
from heapq import heappop, heappush, heapify
from Preprocessing import tokenizer, data, url, title , content
from IndexStructure import PostingsList, Posting
from Indexing import build_index 



[notice] A new release of pip available: 22.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
def search_query(query, index, k, use_champion_list=True):
    token_query = tokenizer(query)
    query_tf = {}
    query_norm = 0

    for word in token_query:
        query_tf[word] = query_tf.get(word, 0) + 1
        query_norm += 1

    query_norm = sqrt(query_norm)
    for word in query_tf:
        query_tf[word] /= query_norm

    candidate_docs = set()
    for word in query_tf:
        if use_champion_list:
            candidate_docs.update(index.give_champion_list(word))
        else:
            candidate_docs.update(index.get_list_word(word))

    heap = []
    heapify(heap)

    for doc_id in candidate_docs:
        score = sum(index.get_weight(doc_id, word) * query_tf[word] for word in query_tf)
        heappush(heap, (score, doc_id))
        if len(heap) > k:
            heappop(heap)

    top_candidates = sorted(heap, reverse=True, key=lambda x: x[0])
    
    for score, doc_id in top_candidates:
        document = list(data.values())[int(doc_id)]
        print(f"Title: {document['title']} | URL: {document['url']}")
    return top_candidates

In [5]:
index = build_index(content)

Indexing documents: 100%|██████████| 12202/12202 [02:52<00:00, 70.64it/s]


In [6]:
query = input("Enter your query: ")
results = search_query(query, index, 5, use_champion_list=False)

Title: نماینده مجلس: سوءتدبیرهای دولت قبل نباید پای دولت سیزدهم نوشته شود | URL: https://www.farsnews.ir/news/14000818000323/نماینده-مجلس-سوءتدبیرهای-دولت-قبل-نباید-پای-دولت-سیزدهم-نوشته-شود
Title: وقتی که برای برجام صرف شد ظرفیت‌های اقتصادی کشور را معطل کرد | URL: https://www.farsnews.ir/news/14001203000871/وقتی-که-برای-برجام-صرف-شد-ظرفیت‌های-اقتصادی-کشور-را-معطل-کرد
Title: جلسه مجمع باشگاه پرسپولیس لغو شد/مدیرعاملی درویش به تاخیر افتاد | URL: https://www.farsnews.ir/news/14001001000639/جلسه-مجمع-باشگاه-پرسپولیس-لغو-شد-مدیرعاملی-درویش-به-تاخیر-افتاد
Title: رئیس جمهور روز دانشجو به دانشگاه شریف می‌رود | URL: https://www.farsnews.ir/news/14000914000507/رئیس-جمهور-روز-دانشجو-به-دانشگاه-شریف-می‌رود
Title: بررسی مدل‌های اقتصادی کاهش مصرف انرژی در کمیسیون اقتصادی | URL: https://www.farsnews.ir/news/14000811000978/بررسی-مدل‌های-اقتصادی-کاهش-مصرف-انرژی-در-کمیسیون-اقتصادی
